# Overdamped Langevin with Markovian switch

Implementation of one or two particles following overdamped Langevin dynamics with a Markovian switch. The Markovian switch consists of transitions between three states given by a continuous-time Markov state model (MSM). 

In [ ]:
import numpy as np
import msmrd2
from msmrd2.markovModels import continuousTimeMarkovStateModel as ctmsm
from msmrd2.integrators import overdampedLangevinMarkovSwitch as odLangevinMS 

In [ ]:
# Define continuous-time MSM
MSMtype = 0 # matches pytpe
ratematrix = np.array([[-5.0,2.0,3.0],[1.0,-2.0,1.0],[3.3,5.4,-8.7]])
seed = 0 # Seed = -1 used random device as seed
markovModel = ctmsm(MSMtype, ratematrix, seed)
# Assign different diffusion coefficients to the different states
Dlist = np.array([1.0, 3.0, 2.0])
Drotlist = np.array([0.5, 0.2, 0.1])
markovModel.setD(Dlist)
markovModel.setDrot(Drotlist)

In [ ]:
# Particle definition (type,initial state, diffusion coefficients, initial configuration)
ptype = 0 # different types will correspond to different MSMs
pstate = 0
D = Dlist[pstate]
Drot = Drotlist[pstate]
position = np.array([1,1,1])
orientation = np.array([1,0,0,0])
part1 = msmrd2.particle(ptype, pstate, D, Drot, position, orientation)

In [ ]:
# Over-damped Langevin integrator with Markovian switch definition
dt = 0.05 # should be smaller than Gillespie inverse transition rates
seed = 0 # seed = -1 uses random device as seed
bodytype = 'rigidbody'
integrator = odLangevinMS(markovModel, dt, seed, bodytype) 

In [ ]:
# Integrates only one particle and print position (orientation is not printed for brevity)
timesteps = 50
# Need to define a one particle list (note part1 object is not modified when list is modified)
part1list = msmrd2.integrators.particleList([part1])
print('{:<10s}{:<15s}{:<15s}{:<15s}{:<15s}{:<40s}'.format("Iteration", "Time", "State", "D", "Drot", "Position 1"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<15d}{:<15f}{:<15f}{:<40s}'
          .format(i, integrator.clock, part1list[0].state, part1list[0].D, part1list[0].Drot, str(part1list[0].position)))
    integrator.integrate(part1list)

Note different states correspond to different diffusion coefficients

## Integration for two particles:

In [ ]:
# Create list of two particles
position2 = np.array([2,2,2])
part1 = msmrd2.particle(ptype, pstate, D, Drot, position, orientation)
part2 = msmrd2.particle(ptype, pstate, D, Drot, position2, orientation)
partlist = msmrd2.integrators.particleList([part1,part2])
# Create second integrator for list integration
integrator2 = odLangevinMS(markovModel, dt, seed, bodytype) 
# Integrate particle list and print only positions (with integratelist)
t = 0
timesteps = 50
print('{:<10s}{:^10s}{:<40s}{:^15s}{:<30s}'.format("Time", "State 1", "Position 1", "State 2", "Position2"))
for i in range(timesteps):
    print('{:<10f}{:^10d}{:<40s}{:^15d}{:<30s}'
          .format(integrator2.clock, partlist[0].state, str(partlist[0].position), partlist[1].state, str(partlist[1].position)))
    integrator2.integrate(partlist)